Dataset: https://www.kaggle.com/datasets/alessandrasala79/ai-vs-human-generated-dataset

## Requirements:

In [5]:
import torch, os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Suspend warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

## Project Goal:
The goal of the project is to create a deep learning model that can identify AI-generated images and differentiate them from real images. The model should not be too complex, and pre-trained networks will be used if there's a <0.60 F1 score.

In [6]:
# Define the basic network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# CUDA tensor types use GPU instead of 
# CPUs for computation the following  
# function checks whether the GPU is available  
# If the GPU is available,the model 
# uses GPU for computation 
if torch.cuda.is_available(): 
    print("Using CUDA")
    model = model.cuda() 
    criterion = criterion.cuda() 
else:
    print("Not using CUDA")
  
print(model) 

Using CUDA
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with image paths and labels.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = self.data_frame.iloc[idx, 0]  # Assuming first column has image paths
        image = Image.open(img_path).convert('RGB')
        label = self.data_frame.iloc[idx, 1]  # Assuming second column has labels
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

def get_data_loaders(csv_path, batch_size=32):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to input size expected by model
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
    ])
    
    # Create dataset
    dataset = ImageDataset(csv_file=csv_path, transform=transform)
    
    # Split into train and validation sets (80/20 split as an example)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader